In [1]:
pip install pandas ta

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
from ta.trend import IchimokuIndicator
from ta.volatility import BollingerBands

# ===== Step 1: Load and Prepare Data =====

file_path = 'C:\\Users\\bajacob\\OneDrive - Tecnicas Reunidas, S.A\\sandbox\\project_2\\Trading_bot\\stock_data\\Adani enterprise 30 min.xlsx'

# Read file, find header, then load actual data
raw_df = pd.read_excel(file_path, header=None)
header_row = raw_df[raw_df[0] == 'Local Date'].index[0]
df = pd.read_excel(file_path, header=header_row)

df['datetime'] = pd.to_datetime(df['Local Date'], errors='coerce')
df.dropna(subset=['datetime'], inplace=True)
df.set_index('datetime', inplace=True)

# Standardize column names
df.rename(columns={
    'Open': 'open', 'High': 'high', 'Low': 'low',
    'Close': 'close', 'Volume': 'volume'
}, inplace=True)

# Cast to numeric and drop bad rows
for col in ['open', 'high', 'low', 'close', 'volume']:
    df[col] = pd.to_numeric(df[col], errors='coerce')
df.dropna(subset=['open', 'high', 'low', 'close', 'volume'], inplace=True)
df.sort_index(inplace=True)

# Keep only necessary columns
expected_cols = ['open', 'high', 'low', 'close', 'volume']
df = df[expected_cols].apply(pd.to_numeric, errors='coerce')
df.dropna(inplace=True)

# ===== Step 2: Technical Indicators =====

df['MA44'] = df['close'].rolling(window=44).mean()

bb = BollingerBands(close=df['close'], window=20, window_dev=2)
df['bb_bbm'] = bb.bollinger_mavg()
df['bb_bbh'] = bb.bollinger_hband()
df['bb_bbl'] = bb.bollinger_lband()

ich = IchimokuIndicator(high=df['high'], low=df['low'])
df['ichimoku_a'] = ich.ichimoku_a()
df['ichimoku_b'] = ich.ichimoku_b()
df['ichimoku_base'] = ich.ichimoku_base_line()
df['ichimoku_conv'] = ich.ichimoku_conversion_line()

df.dropna(subset=[
    'MA44', 'bb_bbm', 'bb_bbh', 'bb_bbl',
    'ichimoku_a', 'ichimoku_b', 'ichimoku_base', 'ichimoku_conv'
], inplace=True)

# ===== Step 3: Trade Direction Logic =====

def get_trade_direction(row):
    if row['close'] > row['MA44'] and row['ichimoku_conv'] > row['ichimoku_base']:
        return 'Long'
    elif row['close'] < row['MA44'] and row['ichimoku_conv'] < row['ichimoku_base']:
        return 'Short'
    else:
        return None

df['trade_type'] = df.apply(get_trade_direction, axis=1)
df['signal'] = df['trade_type'].map({'Long': 'Buy', 'Short': 'Sell'})

def reasoning(row):
    if row['trade_type'] == 'Long':
        return "Close > MA44 and Ichimoku Conversion Line > Base Line (Bullish)"
    elif row['trade_type'] == 'Short':
        return "Close < MA44 and Ichimoku Conversion Line < Base Line (Bearish)"
    else:
        return None

df['reasoning'] = df.apply(reasoning, axis=1)

# ===== Step 4: Stop Loss & Target Strategy =====

df['entry_price'] = df['close']
risk_pct = 0.02
reward_pct = 0.04

df['stop_loss'] = df.apply(lambda r: r['entry_price'] * (1 - risk_pct) if r['trade_type'] == 'Long'
                        else r['entry_price'] * (1 + risk_pct) if r['trade_type'] == 'Short'
                        else None, axis=1)
df['target'] = df.apply(lambda r: r['entry_price'] * (1 + reward_pct) if r['trade_type'] == 'Long'
                     else r['entry_price'] * (1 - reward_pct) if r['trade_type'] == 'Short'
                     else None, axis=1)

# ===== Step 5: Trade Outcome Simulation =====

def simulate_trade_outcome(df, lookahead=30):
    outcomes = []
    profits = []
    for i in range(len(df) - lookahead):
        trade_type = df['trade_type'].iloc[i]
        if not trade_type:
            outcomes.append(None)
            profits.append(None)
            continue

        entry = df['entry_price'].iloc[i]
        stop_loss = df['stop_loss'].iloc[i]
        target = df['target'].iloc[i]
        lows = df['low'].iloc[i+1:i+1+lookahead]
        highs = df['high'].iloc[i+1:i+1+lookahead]
        
        if trade_type == 'Long':
            sl_hit = lows.min() <= stop_loss
            tp_hit = highs.max() >= target
        else:  # Short
            sl_hit = highs.max() >= stop_loss
            tp_hit = lows.min() <= target

        if sl_hit and tp_hit:
            outcomes.append('Both Hit')
            profits.append(((target - entry) / entry) * 100 if tp_hit else ((stop_loss - entry) / entry) * 100)
        elif sl_hit:
            outcomes.append('Stop Loss Hit')
            profits.append(((stop_loss - entry) / entry) * 100 if trade_type == 'Long' else ((entry - stop_loss) / entry) * 100)
        elif tp_hit:
            outcomes.append('Target Hit')
            profits.append(((target - entry) / entry) * 100 if trade_type == 'Long' else ((entry - target) / entry) * 100)
        else:
            outcomes.append('No Trigger')
            profits.append(0)

    outcomes += [None] * lookahead
    profits += [None] * lookahead
    return outcomes, profits

df['trade_outcome'], df['profit_pct'] = simulate_trade_outcome(df)

In [2]:
df

,open,high,low,close,volume,MA44,bb_bbm,bb_bbh,bb_bbl,ichimoku_a,...,ichimoku_base,ichimoku_conv,trade_type,signal,reasoning,entry_price,stop_loss,target,trade_outcome,profit_pct
datetime,,,,,,,,,,,,,,,,,,,,,
2024-08-08 05:30:00,3185.00,3190.3,3168.00,3169.65,175996.0,3110.137500,3145.3025,3218.758327,3071.846673,3153.1375,...,3130.775,3175.500,Long,Buy,Close > MA44 and Ichimoku Conversion Line > Ba...,3169.65,3106.257,3296.436,Stop Loss Hit,-2.0
2024-08-08 06:00:00,3169.60,3194.6,3162.10,3192.30,259477.0,3112.819318,3150.4725,3221.847198,3079.097802,3153.4375,...,3130.775,3176.100,Long,Buy,Close > MA44 and Ichimoku Conversion Line > Ba...,3192.30,3128.454,3319.992,Stop Loss Hit,-2.0
2024-08-08 06:30:00,3192.00,3226.4,3186.15,3206.00,286073.0,3114.673864,3156.6200,3224.815912,3088.424088,3164.8625,...,3142.200,3187.525,Long,Buy,Close > MA44 and Ichimoku Conversion Line > Ba...,3206.00,3141.880,3334.240,Stop Loss Hit,-2.0
2024-08-08 07:00:00,3206.65,3219.8,3196.80,3199.20,91860.0,3116.903409,3162.6675,3222.988585,3102.346415,3166.4500,...,3142.200,3190.700,Long,Buy,Close > MA44 and Ichimoku Conversion Line > Ba...,3199.20,3135.216,3327.168,Stop Loss Hit,-2.0
2024-08-08 07:30:00,3198.10,3212.8,3193.65,3202.00,65450.0,3120.231818,3169.1325,3215.630105,3122.634895,3168.2250,...,3142.200,3194.250,Long,Buy,Close > MA44 and Ichimoku Conversion Line > Ba...,3202.00,3137.960,3330.080,Stop Loss Hit,-2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-08-04 09:30:00,2362.00,2372.0,2361.60,2371.40,19205.0,2433.820455,2372.8150,2417.427701,2328.202299,2372.4750,...,2389.800,2355.150,Short,Sell,Close < MA44 and Ichimoku Conversion Line < Ba...,2371.40,2418.828,2276.544,None,NaN
2025-08-04 10:00:00,2371.30,2374.7,2362.20,2362.40,38661.0,2429.793182,2369.8400,2408.501578,2331.178422,2375.1250,...,2389.800,2360.450,Short,Sell,Close < MA44 and Ichimoku Conversion Line < Ba...,2362.40,2409.648,2267.904,None,NaN
2025-08-04 10:30:00,2362.00,2368.7,2358.40,2364.00,42353.0,2425.759091,2367.4600,2401.076984,2333.843016,2375.1250,...,2389.800,2360.450,Short,Sell,Close < MA44 and Ichimoku Conversion Line < Ba...,2364.00,2411.280,2269.440,None,NaN


In [3]:
df

,open,high,low,close,volume,MA44,bb_bbm,bb_bbh,bb_bbl,ichimoku_a,...,ichimoku_base,ichimoku_conv,trade_type,signal,reasoning,entry_price,stop_loss,target,trade_outcome,profit_pct
datetime,,,,,,,,,,,,,,,,,,,,,
2024-08-08 05:30:00,3185.00,3190.3,3168.00,3169.65,175996.0,3110.137500,3145.3025,3218.758327,3071.846673,3153.1375,...,3130.775,3175.500,Long,Buy,Close > MA44 and Ichimoku Conversion Line > Ba...,3169.65,3106.257,3296.436,Stop Loss Hit,-2.0
2024-08-08 06:00:00,3169.60,3194.6,3162.10,3192.30,259477.0,3112.819318,3150.4725,3221.847198,3079.097802,3153.4375,...,3130.775,3176.100,Long,Buy,Close > MA44 and Ichimoku Conversion Line > Ba...,3192.30,3128.454,3319.992,Stop Loss Hit,-2.0
2024-08-08 06:30:00,3192.00,3226.4,3186.15,3206.00,286073.0,3114.673864,3156.6200,3224.815912,3088.424088,3164.8625,...,3142.200,3187.525,Long,Buy,Close > MA44 and Ichimoku Conversion Line > Ba...,3206.00,3141.880,3334.240,Stop Loss Hit,-2.0
2024-08-08 07:00:00,3206.65,3219.8,3196.80,3199.20,91860.0,3116.903409,3162.6675,3222.988585,3102.346415,3166.4500,...,3142.200,3190.700,Long,Buy,Close > MA44 and Ichimoku Conversion Line > Ba...,3199.20,3135.216,3327.168,Stop Loss Hit,-2.0
2024-08-08 07:30:00,3198.10,3212.8,3193.65,3202.00,65450.0,3120.231818,3169.1325,3215.630105,3122.634895,3168.2250,...,3142.200,3194.250,Long,Buy,Close > MA44 and Ichimoku Conversion Line > Ba...,3202.00,3137.960,3330.080,Stop Loss Hit,-2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-08-04 09:30:00,2362.00,2372.0,2361.60,2371.40,19205.0,2433.820455,2372.8150,2417.427701,2328.202299,2372.4750,...,2389.800,2355.150,Short,Sell,Close < MA44 and Ichimoku Conversion Line < Ba...,2371.40,2418.828,2276.544,None,NaN
2025-08-04 10:00:00,2371.30,2374.7,2362.20,2362.40,38661.0,2429.793182,2369.8400,2408.501578,2331.178422,2375.1250,...,2389.800,2360.450,Short,Sell,Close < MA44 and Ichimoku Conversion Line < Ba...,2362.40,2409.648,2267.904,None,NaN
2025-08-04 10:30:00,2362.00,2368.7,2358.40,2364.00,42353.0,2425.759091,2367.4600,2401.076984,2333.843016,2375.1250,...,2389.800,2360.450,Short,Sell,Close < MA44 and Ichimoku Conversion Line < Ba...,2364.00,2411.280,2269.440,None,NaN


### Latest code

In [3]:
import pandas as pd
from ta.trend import IchimokuIndicator

# ===== Step 1: Load and Prepare Data =====

file_path = 'C:\\Users\\bajacob\\OneDrive - Tecnicas Reunidas, S.A\\sandbox\\project_2\\Trading_bot\\stock_data\\Adani enterprise 30 min.xlsx'

# Read file, find header, then load actual data
raw_df = pd.read_excel(file_path, header=None)
header_row = raw_df[raw_df[0] == 'Local Date'].index[0]
df = pd.read_excel(file_path, header=header_row)

df['datetime'] = pd.to_datetime(df['Local Date'], errors='coerce')
df.dropna(subset=['datetime'], inplace=True)
df.set_index('datetime', inplace=True)

# Standardize column names
df.rename(columns={
    'Open': 'open', 'High': 'high', 'Low': 'low',
    'Close': 'close', 'Volume': 'volume'
}, inplace=True)

# Cast to numeric and drop bad rows
for col in ['open', 'high', 'low', 'close', 'volume']:
    df[col] = pd.to_numeric(df[col], errors='coerce')
df.dropna(subset=['open', 'high', 'low', 'close', 'volume'], inplace=True)
df.sort_index(inplace=True)

# Keep only necessary columns
expected_cols = ['open', 'high', 'low', 'close', 'volume']
df = df[expected_cols].apply(pd.to_numeric, errors='coerce')
df.dropna(inplace=True)

# ===== Step 2: Ichimoku Indicator (Pure Ichimoku) =====
# Some versions of ta.IchimokuIndicator do not accept 'close' kwarg; it only uses high/low.
# Also set parameters explicitly for clarity.
ich = IchimokuIndicator(
    high=df['high'],
    low=df['low'],
    window1=9,     # Tenkan
    window2=26,    # Kijun
    window3=52,    # Senkou B
    visual=False   # we want lines aligned to current index, not forward/back shifted
)

df['tenkan_sen'] = ich.ichimoku_conversion_line()
df['kijun_sen']   = ich.ichimoku_base_line()
df['senkou_span_a'] = ich.ichimoku_a()
df['senkou_span_b'] = ich.ichimoku_b()

# Chikou: price plotted 26 periods back. We store the value aligned with current index for logic.
df['chikou_span'] = df['close'].shift(-26)

# Cloud boundaries
df['cloud_top'] = df[['senkou_span_a', 'senkou_span_b']].max(axis=1)
df['cloud_bottom'] = df[['senkou_span_a', 'senkou_span_b']].min(axis=1)

# Drop rows where Ichimoku needs warmup
df.dropna(subset=['tenkan_sen', 'kijun_sen', 'senkou_span_a', 'senkou_span_b'], inplace=True)

# ===== Step 3: Ichimoku Signal Logic =====
def get_ichimoku_signal(row, idx, df_full):
    price = row['close']
    tenkan = row['tenkan_sen']
    kijun = row['kijun_sen']
    cloud_top = row['cloud_top']
    cloud_bottom = row['cloud_bottom']

    # Chikou confirmation: compare current price to price 26 periods ago (index-safe)
    if idx >= 26:
        chikou_price_26_ago = df_full['close'].iloc[idx-26]
        chikou_bullish = price > chikou_price_26_ago
    else:
        chikou_bullish = None

    # Future cloud color (26 periods ahead, index-safe)
    if idx + 26 < len(df_full):
        future_senkou_a = df_full['senkou_span_a'].iloc[idx+26]
        future_senkou_b = df_full['senkou_span_b'].iloc[idx+26]
        future_cloud_bullish = future_senkou_a > future_senkou_b
    else:
        future_cloud_bullish = row['senkou_span_a'] > row['senkou_span_b']

    price_above_cloud = price > cloud_top
    price_below_cloud = price < cloud_bottom
    tenkan_above_kijun = tenkan > kijun
    tenkan_below_kijun = tenkan < kijun

    # Strong/Weak signals
    if price_above_cloud and tenkan_above_kijun and (chikou_bullish is None or chikou_bullish) and future_cloud_bullish:
        return 'Strong_Long'
    if price_below_cloud and tenkan_below_kijun and (chikou_bullish is None or not chikou_bullish) and (not future_cloud_bullish):
        return 'Strong_Short'
    if price_above_cloud and tenkan_above_kijun:
        return 'Weak_Long'
    if price_below_cloud and tenkan_below_kijun:
        return 'Weak_Short'
    if cloud_bottom <= price <= cloud_top:
        return 'Inside_Cloud'
    return 'No_Signal'

signals = []
for idx, (_, row) in enumerate(df.iterrows()):
    signals.append(get_ichimoku_signal(row, idx, df))
df['signal_type'] = signals
df['trade_signal'] = df['signal_type'].map({
    'Strong_Long': 'BUY',
    'Weak_Long': 'BUY',
    'Strong_Short': 'SELL',
    'Weak_Short': 'SELL'
})

# ===== Step 4: Dynamic Stop Loss (Ichimoku) & 1:2 Target =====
def calculate_ichimoku_levels(row):
    price = row['close']
    signal = row['signal_type']
    kijun = row['kijun_sen']
    cloud_top = row['cloud_top']
    cloud_bottom = row['cloud_bottom']

    if 'Long' in signal:
        stop_loss = max(kijun, cloud_top)
        risk = price - stop_loss
        if risk <= 0:
            return (pd.NA, pd.NA)  # invalid levels; skip trade
        target = price + 2 * risk  # 1:2 RR
        return (stop_loss, target)

    if 'Short' in signal:
        stop_loss = min(kijun, cloud_bottom)
        risk = stop_loss - price
        if risk <= 0:
            return (pd.NA, pd.NA)  # invalid levels; skip trade
        target = price - 2 * risk
        return (stop_loss, target)

    return (pd.NA, pd.NA)

levels = df.apply(calculate_ichimoku_levels, axis=1, result_type='expand')
df['stop_loss'] = levels[0]
df['target'] = levels[1]             # FIXED: was levels[1]
df['entry_price'] = df['close']

# ===== Step 5: Trade Outcome Simulation (Profit %) =====
def simulate_ichimoku_trades(df, lookahead=30):
    outcomes = []
    profit_pct = []
    n = len(df)

    for i in range(n - lookahead):
        side = df['trade_signal'].iloc[i]
        entry = df['entry_price'].iloc[i]
        stop_loss = df['stop_loss'].iloc[i]
        target = df['target'].iloc[i]

        if side not in ['BUY', 'SELL'] or pd.isna(stop_loss) or pd.isna(target):
            outcomes.append(None)
            profit_pct.append(None)
            continue

        future = df.iloc[i+1:i+1+lookahead]

        if side == 'BUY':
            sl_hits = future[future['low'] <= stop_loss]
            tp_hits = future[future['high'] >= target]
            sl_hit = not sl_hits.empty
            tp_hit = not tp_hits.empty
            if sl_hit and tp_hit:
                first_sl_time = sl_hits.index[0]
                first_tp_time = tp_hits.index     # FIXED: use index for first time
                if first_sl_time <= first_tp_time:
                    outcomes.append('Stop_Loss_Hit')
                    profit_pct.append((stop_loss - entry) / entry * 100)
                else:
                    outcomes.append('Target_Hit')
                    profit_pct.append((target - entry) / entry * 100)
            elif sl_hit:
                outcomes.append('Stop_Loss_Hit')
                profit_pct.append((stop_loss - entry) / entry * 100)
            elif tp_hit:
                outcomes.append('Target_Hit')
                profit_pct.append((target - entry) / entry * 100)
            else:
                outcomes.append('No_Exit')
                last = future['close'].iloc[-1]
                profit_pct.append((last - entry) / entry * 100)

        else:  # SELL
            sl_hits = future[future['high'] >= stop_loss]
            tp_hits = future[future['low'] <= target]
            sl_hit = not sl_hits.empty
            tp_hit = not tp_hits.empty
            if sl_hit and tp_hit:
                first_sl_time = sl_hits.index[0]
                first_tp_time = tp_hits.index     # FIXED: use index
                if first_sl_time <= first_tp_time:
                    outcomes.append('Stop_Loss_Hit')
                    profit_pct.append((entry - stop_loss) / entry * 100)
                else:
                    outcomes.append('Target_Hit')
                    profit_pct.append((entry - target) / entry * 100)
            elif sl_hit:
                outcomes.append('Stop_Loss_Hit')
                profit_pct.append((entry - stop_loss) / entry * 100)
            elif tp_hit:
                outcomes.append('Target_Hit')
                profit_pct.append((entry - target) / entry * 100)
            else:
                outcomes.append('No_Exit')
                last = future['close'].iloc[-1]
                profit_pct.append((entry - last) / entry * 100)

    # pad tail
    outcomes += [None] * lookahead
    profit_pct += [None] * lookahead
    return outcomes, profit_pct

df['trade_outcome'], df['profit_pct'] = simulate_ichimoku_trades(df, lookahead=30)

# Optional: keep only actionable rows
# trades = df[df['trade_signal'].isin(['BUY', 'SELL'])].copy()
# print(trades[['close','signal_type','trade_signal','stop_loss','target','trade_outcome','profit_pct']].tail(20))


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [ ]:
import pandas as pd
from ta.trend import IchimokuIndicator

# =========================
# Step 1: Load and Prepare
# =========================

file_path = 'C:\\Users\\bajacob\\OneDrive - Tecnicas Reunidas, S.A\\sandbox\\project_2\\Trading_bot\\stock_data\\Adani enterprise 30 min.xlsx'

# Read raw without header
raw_df = pd.read_excel(file_path, header=None, dtype=str)

# Vectorized strip for object columns (avoid applymap deprecation)
raw_norm = raw_df.copy()
obj_cols = raw_norm.select_dtypes(include=['object']).columns
raw_norm[obj_cols] = raw_norm[obj_cols].apply(lambda s: s.str.strip() if s.dtype == 'object' else s)

# Find the header row (any cell equals 'Local Date', case-insensitive)
matches = raw_norm.apply(lambda row: row.astype(str).str.lower().str.strip().eq('local date').any(), axis=1)
if not matches.any():
    raise ValueError("Could not find a header row containing 'Local Date' in the Excel file.")

header_row = int(matches[matches].index[0])  # ensure int for pandas

# Load data using detected header row
df = pd.read_excel(file_path, header=header_row)
df.columns = [str(c).strip() for c in df.columns]

# Basic datetime index setup
df['datetime'] = pd.to_datetime(df['Local Date'], errors='coerce')
df.dropna(subset=['datetime'], inplace=True)
df.set_index('datetime', inplace=True)

# Standardize column names
df.rename(columns={
    'Open': 'open', 'High': 'high', 'Low': 'low',
    'Close': 'close', 'Volume': 'volume'
}, inplace=True)

# Numeric casting and cleaning
for col in ['open', 'high', 'low', 'close', 'volume']:
    df[col] = pd.to_numeric(df[col], errors='coerce')
df.dropna(subset=['open', 'high', 'low', 'close', 'volume'], inplace=True)
df.sort_index(inplace=True)

# Keep only needed columns
df = df[['open', 'high', 'low', 'close', 'volume']].copy()

# =====================================
# Step 2: Ichimoku Indicator (Pure Ichi)
# =====================================

# Some library versions use only high/low (no close kwarg). This call is version-agnostic.
ich = IchimokuIndicator(
    high=df['high'],
    low=df['low'],
    window1=9,    # Tenkan
    window2=26,   # Kijun
    window3=52,   # Senkou B
    visual=False  # keep lines aligned to current index
)

df['tenkan_sen']    = ich.ichimoku_conversion_line()
df['kijun_sen']     = ich.ichimoku_base_line()
df['senkou_span_a'] = ich.ichimoku_a()
df['senkou_span_b'] = ich.ichimoku_b()

# Chikou: current price plotted 26 periods back
df['chikou_span'] = df['close'].shift(-26)

# Cloud boundaries
df['cloud_top'] = df[['senkou_span_a', 'senkou_span_b']].max(axis=1)
df['cloud_bottom'] = df[['senkou_span_a', 'senkou_span_b']].min(axis=1)

# Drop warmup rows
df.dropna(subset=['tenkan_sen', 'kijun_sen', 'senkou_span_a', 'senkou_span_b'], inplace=True)

# ======================================
# Step 3: Ichimoku Multi-Confirm Signals
# ======================================

def get_ichimoku_signal(row, idx, df_full):
    price = row['close']
    tenkan = row['tenkan_sen']
    kijun = row['kijun_sen']
    cloud_top = row['cloud_top']
    cloud_bottom = row['cloud_bottom']

    # Chikou confirmation vs price 26 periods ago
    if idx >= 26:
        chikou_price_26_ago = df_full['close'].iloc[idx-26]
        chikou_bullish = price > chikou_price_26_ago
    else:
        chikou_bullish = None

    # Future cloud color 26 periods ahead
    if idx + 26 < len(df_full):
        future_senkou_a = df_full['senkou_span_a'].iloc[idx+26]
        future_senkou_b = df_full['senkou_span_b'].iloc[idx+26]
        future_cloud_bullish = future_senkou_a > future_senkou_b
    else:
        future_cloud_bullish = row['senkou_span_a'] > row['senkou_span_b']

    price_above_cloud = price > cloud_top
    price_below_cloud = price < cloud_bottom
    tenkan_above_kijun = tenkan > kijun
    tenkan_below_kijun = tenkan < kijun

    # Strong signals (with chikou and future-cloud confirmation)
    if price_above_cloud and tenkan_above_kijun and (chikou_bullish is None or chikou_bullish) and future_cloud_bullish:
        return 'Strong_Long'
    if price_below_cloud and tenkan_below_kijun and (chikou_bullish is None or not chikou_bullish) and (not future_cloud_bullish):
        return 'Strong_Short'

    # Weak signals (partial confirmation)
    if price_above_cloud and tenkan_above_kijun:
        return 'Weak_Long'
    if price_below_cloud and tenkan_below_kijun:
        return 'Weak_Short'

    # Inside the cloud -> wait
    if cloud_bottom <= price <= cloud_top:
        return 'Inside_Cloud'

    return 'No_Signal'

signals = []
for idx, (_, row) in enumerate(df.iterrows()):
    signals.append(get_ichimoku_signal(row, idx, df))
df['signal_type'] = signals
df['trade_signal'] = df['signal_type'].map({
    'Strong_Long': 'BUY',
    'Weak_Long': 'BUY',
    'Strong_Short': 'SELL',
    'Weak_Short': 'SELL'
})

# ==========================================
# Step 4: Dynamic SL (Ichimoku) & 1:2 Target
# ==========================================

def calculate_ichimoku_levels(row):
    price = row['close']
    signal = row['signal_type']
    kijun = row['kijun_sen']
    cloud_top = row['cloud_top']
    cloud_bottom = row['cloud_bottom']

    # Long: SL at stronger of Kijun vs Cloud top (support). Target at 1:2 RR.
    if isinstance(signal, str) and 'Long' in signal:
        stop_loss = max(kijun, cloud_top)
        risk = price - stop_loss
        if risk <= 0:
            return (pd.NA, pd.NA)
        target = price + 2 * risk
        return (stop_loss, target)

    # Short: SL at stronger of Kijun vs Cloud bottom (resistance). Target at 1:2 RR.
    if isinstance(signal, str) and 'Short' in signal:
        stop_loss = min(kijun, cloud_bottom)
        risk = stop_loss - price
        if risk <= 0:
            return (pd.NA, pd.NA)
        target = price - 2 * risk
        return (stop_loss, target)

    return (pd.NA, pd.NA)

levels = df.apply(calculate_ichimoku_levels, axis=1, result_type='expand')
df['stop_loss'] = levels[0]
df['target'] = levels[1]
df['entry_price'] = df['close']

# =====================================
# Step 5: Simulation with Profit Percent
# =====================================

def simulate_ichimoku_trades(df, lookahead=30):
    outcomes = []
    profit_pct = []
    n = len(df)

    for i in range(n - lookahead):
        side = df['trade_signal'].iloc[i]
        entry = df['entry_price'].iloc[i]
        stop_loss = df['stop_loss'].iloc[i]
        target = df['target'].iloc[i]

        # Skip invalid/non-trade rows
        if side not in ['BUY', 'SELL'] or pd.isna(stop_loss) or pd.isna(target):
            outcomes.append(None)
            profit_pct.append(None)
            continue

        future = df.iloc[i+1:i+1+lookahead]

        if side == 'BUY':
            sl_hits = future[future['low'] <= stop_loss]
            tp_hits = future[future['high'] >= target]
            sl_hit = not sl_hits.empty
            tp_hit = not tp_hits.empty
            if sl_hit and tp_hit:
                first_sl_time = sl_hits.index[0]     # FIRST timestamp -> scalar compare
                first_tp_time = tp_hits.index[0]     # FIXED: take first timestamp
                if first_sl_time <= first_tp_time:
                    outcomes.append('Stop_Loss_Hit')
                    profit_pct.append((stop_loss - entry) / entry * 100)
                else:
                    outcomes.append('Target_Hit')
                    profit_pct.append((target - entry) / entry * 100)
            elif sl_hit:
                outcomes.append('Stop_Loss_Hit')
                profit_pct.append((stop_loss - entry) / entry * 100)
            elif tp_hit:
                outcomes.append('Target_Hit')
                profit_pct.append((target - entry) / entry * 100)
            else:
                outcomes.append('No_Exit')
                last = future['close'].iloc[-1]
                profit_pct.append((last - entry) / entry * 100)

        else:  # SELL
            sl_hits = future[future['high'] >= stop_loss]
            tp_hits = future[future['low'] <= target]
            sl_hit = not sl_hits.empty
            tp_hit = not tp_hits.empty
            if sl_hit and tp_hit:
                first_sl_time = sl_hits.index[0]
                first_tp_time = tp_hits.index[0]     # FIXED: take first timestamp
                if first_sl_time <= first_tp_time:
                    outcomes.append('Stop_Loss_Hit')
                    profit_pct.append((entry - stop_loss) / entry * 100)
                else:
                    outcomes.append('Target_Hit')
                    profit_pct.append((entry - target) / entry * 100)
            elif sl_hit:
                outcomes.append('Stop_Loss_Hit')
                profit_pct.append((entry - stop_loss) / entry * 100)
            elif tp_hit:
                outcomes.append('Target_Hit')
                profit_pct.append((entry - target) / entry * 100)
            else:
                outcomes.append('No_Exit')
                last = future['close'].iloc[-1]
                profit_pct.append((entry - last) / entry * 100)

    # pad tail for alignment
    outcomes += [None] * lookahead
    profit_pct += [None] * lookahead
    return outcomes, profit_pct



# Optional: show last few actionable rows
# trades = df[df['trade_signal'].isin(['BUY', 'SELL'])].copy()
# print(trades[['close','signal_type','trade_signal','stop_loss','target','trade_outcome','profit_pct']].tail(20))


Sample trades (last 10):
                     entry_price   close   signal_type trade_signal stop_loss  \
datetime                                                                        
2025-08-04 07:00:00       2357.5  2357.5  Strong_Short         SELL   2421.45   
2025-08-04 07:30:00       2359.1  2359.1  Strong_Short         SELL   2421.05   
2025-08-04 08:00:00       2368.7  2367.5  Strong_Short         SELL    2414.6   
2025-08-04 08:30:00       2366.0  2366.7  Strong_Short         SELL    2411.5   
2025-08-04 09:00:00       2362.0  2362.0  Strong_Short         SELL   2407.85   
2025-08-04 09:30:00       2371.3  2371.4  Strong_Short         SELL    2389.8   
2025-08-04 10:00:00       2362.0  2362.4  Strong_Short         SELL    2389.8   
2025-08-04 10:30:00       2363.8  2364.0  Strong_Short         SELL    2389.8   
2025-08-04 11:00:00       2364.7  2364.7  Strong_Short         SELL    2389.8   
2025-08-04 11:30:00          NaN  2363.6  Strong_Short         SELL   2383.55   

  

In [11]:
# Diagnostic & robust loader for notebook (paste & run)
import os, math
import pandas as pd
import numpy as np
from ta.trend import IchimokuIndicator
import matplotlib.pyplot as plt

FILE = r"C:\Users\bajacob\OneDrive - Tecnicas Reunidas, S.A\sandbox\project_2\Trading_bot\stock_data\Adani enterprise 5 min.xlsx"

def read_raw_any(path):
    ext = os.path.splitext(path)[1].lower()
    if ext in (".csv", ".txt"):
        raw = pd.read_csv(path, header=None, dtype=str, engine="python", error_bad_lines=False)
    else:
        # try excel read without header to inspect rows
        try:
            raw = pd.read_excel(path, header=None, dtype=str, engine="openpyxl")
        except Exception:
            try:
                import pyxlsb
                raw = pd.read_excel(path, header=None, dtype=str, engine="pyxlsb")
            except Exception:
                raw = pd.read_excel(path, header=None, dtype=str)
    return raw

def detect_header_row(raw):
    rawf = raw.fillna("").astype(str)
    for i, row in rawf.iterrows():
        low = row.str.strip().str.lower()
        if (low == "local date").any() or (low == "localdate").any():
            return int(i)
    # try other heuristics: any row that contains 'local' and 'date' tokens
    for i, row in rawf.iterrows():
        s = " ".join(row.values).lower()
        if "local" in s and "date" in s:
            return int(i)
    return None

def try_parse_datetime(series, formats):
    # returns tuple (parsed_series, successful_count, used_format_or_None)
    for fmt in formats:
        try:
            parsed = pd.to_datetime(series, format=fmt, errors="coerce")
            ok = parsed.notna().sum()
            if ok > 0:
                return parsed, ok, fmt
        except Exception:
            continue
    # fallback to pandas auto (dateutil) — may be slower but sometimes needed
    parsed = pd.to_datetime(series, errors="coerce")
    return parsed, parsed.notna().sum(), None

def load_and_debug(path):
    print("Reading raw file (no header) to inspect initial rows...")
    raw = read_raw_any(path)
    print(f"Raw shape: {raw.shape}")
    display(raw.head(10))
    header_row = detect_header_row(raw)
    print("Detected header row:", header_row)
    # if header_row found, read with header
    if header_row is not None:
        try:
            df = pd.read_excel(path, header=header_row, engine="openpyxl")
        except Exception:
            try:
                df = pd.read_excel(path, header=header_row)
            except Exception:
                # fallback use raw and promote header
                df = raw.copy()
                df.columns = df.iloc[header_row].tolist()
                df = df.drop(index=list(range(0, header_row+1)))
    else:
        # try normal read with inference
        try:
            df = pd.read_excel(path)
        except Exception:
            try:
                df = pd.read_csv(path)
            except Exception:
                # use raw and promote first row as header (best effort)
                df = raw.copy()
                df.columns = raw.iloc[0].tolist()
                df = df.drop(index=0)

    # normalize column names
    orig_cols = list(df.columns)
    print("Columns found (raw):", orig_cols)
    cols_norm = {c: str(c).strip() for c in orig_cols}
    df.rename(columns=cols_norm, inplace=True)

    # try to find likely column names
    colnames = [c.strip().lower() for c in df.columns]
    mapping = {}
    def find_like(targets):
        for t in targets:
            for c in df.columns:
                if t in c.strip().lower():
                    return c
        return None

    candidates = {
        "local_date": find_like(["local date","localdate","date","day"]),
        "local_time": find_like(["local time","time","timestamp"]),
        "open": find_like(["open"]),
        "high": find_like(["high","hi"]),
        "low": find_like(["low","lo"]),
        "close": find_like(["close","last","ltp","price"]),
        "volume": find_like(["volume","vol"])
    }
    print("Column detection candidates:", candidates)

    # Build datetime column string
    if candidates["local_date"]:
        if candidates["local_time"]:
            dt_series = df[candidates["local_date"]].astype(str).str.strip() + " " + df[candidates["local_time"]].astype(str).str.strip()
        else:
            dt_series = df[candidates["local_date"]].astype(str).str.strip()
    else:
        print("ERROR: could not locate a date column automatically. Columns:", df.columns.tolist())
        display(df.head(20))
        raise ValueError("Missing Local Date column")

    # Attempt multiple datetime formats (common ones for your region)
    formats_to_try = [
        "%Y-%m-%d %H:%M:%S", "%Y-%m-%d %H:%M",
        "%d-%m-%Y %H:%M:%S", "%d-%m-%Y %H:%M",
        "%d/%m/%Y %H:%M:%S", "%d/%m/%Y %H:%M",
        "%d-%b-%Y %H:%M", "%d %b %Y %H:%M",
        "%m/%d/%Y %H:%M:%S","%m/%d/%Y %H:%M",
        "%Y/%m/%d %H:%M", "%Y.%m.%d %H:%M",
        "%Y%m%d %H%M", "%Y%m%d%H%M"
    ]
    parsed, ok_count, used_fmt = try_parse_datetime(dt_series, formats_to_try)
    print(f"Datetime parse success count: {ok_count} / {len(dt_series)}  (used format: {used_fmt})")
    if ok_count < max(10, len(dt_series)*0.01):  # less than 1% or less than 10 rows parsed
        print("Few datetimes parsed successfully — showing sample of date strings for inspection:")
        display(dt_series.head(30))
        # attempt to show unique separators and examples
        uniques = pd.Series(dt_series.dropna().unique()[:20])
        print("Unique datetime samples (up to 20):")
        display(uniques)
    # set datetime index
    df["_dt_parsed_"] = parsed
    df = df.dropna(subset=["_dt_parsed_"]).copy()
    df = df.set_index("_dt_parsed_")
    df.index.name = "datetime"
    print("After setting datetime index, shape:", df.shape)
    # Now map OHLCV columns
    def pick(colkeys):
        for k in colkeys:
            for c in df.columns:
                if k in c.strip().lower():
                    return c
        return None

    o_col = pick(["open"])
    h_col = pick(["high"])
    l_col = pick(["low"])
    c_col = pick(["close","last","ltp","price"])
    v_col = pick(["volume","vol"])

    print("Mapped columns ->", {"open":o_col,"high":h_col,"low":l_col,"close":c_col,"volume":v_col})

    if not all([o_col,h_col,l_col,c_col]):
        print("Could not map all OHLC columns. Showing head for debugging:")
        display(df.head(20))
        raise ValueError("Missing OHLC columns after mapping")

    # create canonical df
    df2 = pd.DataFrame(index=df.index)
    df2["open"] = pd.to_numeric(df[o_col], errors="coerce")
    df2["high"] = pd.to_numeric(df[h_col], errors="coerce")
    df2["low"]  = pd.to_numeric(df[l_col], errors="coerce")
    df2["close"] = pd.to_numeric(df[c_col], errors="coerce")
    if v_col:
        df2["volume"] = pd.to_numeric(df[v_col], errors="coerce").fillna(0.0)
    else:
        df2["volume"] = 0.0

    print("Usable OHLC rows before dropping NaNs:", len(df2))
    df2 = df2.dropna(subset=["open","high","low","close"]).sort_index()
    print("Usable OHLC rows after dropping NaNs:", len(df2))
    if df2.empty:
        print("Empty after dropping NaNs. Showing sample of raw mapped columns:")
        display(df[[o_col,h_col,l_col,c_col]].head(30))
        raise ValueError("No usable OHLC rows found after cleaning.")
    return df2

# Run loader & give feedback
df_clean = load_and_debug(FILE)
print("Loaded dataframe index range:", df_clean.index.min(), "->", df_clean.index.max())
display(df_clean.head(5))
display(df_clean.tail(5))

# If loaded OK, continue with a quick ichimoku check (optional)
def quick_ichimoku_check(df):
    ich = IchimokuIndicator(high=df['high'], low=df['low'], window1=9, window2=26, window3=52, visual=False)
    df2 = df.copy()
    df2['tenkan'] = ich.ichimoku_conversion_line()
    df2['kijun'] = ich.ichimoku_base_line()
    df2 = df2.dropna(subset=['tenkan','kijun'])
    print("Ichimoku warmup rows removed, usable rows:", len(df2))
    display(df2[['open','high','low','close','tenkan','kijun']].tail(8))
    return df2

df_check = quick_ichimoku_check(df_clean)


Reading raw file (no header) to inspect initial rows...
Raw shape: (20780, 13)


,0,1,2,3,4,5,6,7,8,9,10,11,12
0,Adani Enterprises Ltd | Price History ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ADEL.NS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Interval: 5 Minutes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,History Period: 04-Aug-2024 13:50 - 03-Aug-202...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,VAP: Total,411138540,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Price,Volume,%Volume,Count,%Count,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Detected header row: 27
Columns found (raw): ['Local Date', 'Local Time', 'Close', 'Net', '%Chg', 'Open', 'Low', 'High', 'Volume', 'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12']
Column detection candidates: {'local_date': 'Local Date', 'local_time': 'Local Time', 'open': 'Open', 'high': 'High', 'low': 'Low', 'close': 'Close', 'volume': 'Volume'}


C:\Users\bajacob\AppData\Local\Temp\ipykernel_13952\33273046.py:50: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  parsed = pd.to_datetime(series, errors="coerce")


Datetime parse success count: 0 / 20752  (used format: None)
Few datetimes parsed successfully — showing sample of date strings for inspection:


...

Unique datetime samples (up to 20):


0     2025-08-01 12:50:00 2025-08-01 12:50:00
1     2025-08-01 11:30:00 2025-08-01 11:30:00
2     2025-08-01 11:25:00 2025-08-01 11:25:00
3     2025-08-01 11:20:00 2025-08-01 11:20:00
4     2025-08-01 11:15:00 2025-08-01 11:15:00
5     2025-08-01 11:05:00 2025-08-01 11:05:00
6     2025-08-01 11:00:00 2025-08-01 11:00:00
7     2025-08-01 10:55:00 2025-08-01 10:55:00
8     2025-08-01 10:50:00 2025-08-01 10:50:00
9     2025-08-01 10:45:00 2025-08-01 10:45:00
10    2025-08-01 10:40:00 2025-08-01 10:40:00
11    2025-08-01 10:35:00 2025-08-01 10:35:00
12    2025-08-01 10:30:00 2025-08-01 10:30:00
13    2025-08-01 10:25:00 2025-08-01 10:25:00
14    2025-08-01 10:20:00 2025-08-01 10:20:00
15    2025-08-01 10:15:00 2025-08-01 10:15:00
16    2025-08-01 10:10:00 2025-08-01 10:10:00
17    2025-08-01 10:05:00 2025-08-01 10:05:00
18    2025-08-01 10:00:00 2025-08-01 10:00:00
19    2025-08-01 09:55:00 2025-08-01 09:55:00
dtype: object

After setting datetime index, shape: (0, 13)
Mapped columns -> {'open': 'Open', 'high': 'High', 'low': 'Low', 'close': 'Close', 'volume': 'Volume'}
Usable OHLC rows before dropping NaNs: 0
Usable OHLC rows after dropping NaNs: 0
Empty after dropping NaNs. Showing sample of raw mapped columns:


,Open,High,Low,Close
datetime,,,,


ValueError: No usable OHLC rows found after cleaning.

In [9]:
pip install pandas numpy matplotlib ta pyxlsb

  Using cached contourpy-1.3.3-cp311-cp311-win_amd64.whl.metadata (5.5 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached kiwisolver-1.4.9-cp311-cp311-win_amd64.whl.metadata (6.4 kB)
   ---------------------------------------- 0.0/8.1 MB ? eta -:--:--
   ----- ---------------------------------- 1.0/8.1 MB 6.3 MB/s eta 0:00:02
   ------------------ --------------------- 3.7/8.1 MB 9.5 MB/s eta 0:00:01
   ------------------------------------- -- 7.6/8.1 MB 12.7 MB/s eta 0:00:01
   ---------------------------------------- 8.1/8.1 MB 12.6 MB/s  0:00:00
Using cached contourpy-1.3.3-cp311-cp311-win_amd64.whl (225 kB)
Using cached cycler-0.12.1-py3-none-any.whl (8.3 kB)
   ---------------------------------------- 0.0/2.3 MB ? eta -:--:--
   ---------------------------------------- 2.3/2.3 MB 21.7 MB/s  0:00:00
Using cached kiwisolver-1.4.9-cp311-cp311-win_amd64.whl (73 kB)

   ---------------------------------------- 0/6 [pyparsing]
   ------------- -----------